In [56]:
%load_ext autoreload 
%autoreload 2
import newspaper
import rag
from feeds import TheGuardian
import feedparser

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from top2vec import Top2Vec

In [55]:

from feeds import AssociatedPress, TheGuardian, BBC, download_feeds

docs = download_feeds([TheGuardian, BBC, AssociatedPress])
len(docs)


256

In [54]:

Top2Vec.contextual_top2vec = False
model = Top2Vec([d.content for d in docs], embedding_model='universal-sentence-encoder', min_count=1, speed="deep-learn", ngram_vocab=False)
num_topics = model.get_num_topics()
topic_words, word_scores, topic_nums = model.get_topics(num_topics)
topic_sizes, topic_nums = model.get_topic_sizes()
for size, words in zip(topic_sizes, topic_words):
    print(f"{size}: {', '.join(words)}")

2024-11-15 12:33:56,652 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
/opt/homebrew/Caskroom/miniforge/base/envs/newsrag/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2024-11-15 12:33:56,661 - top2vec - INFO - Downloading universal-sentence-encoder model
INFO:top2vec:Downloading universal-sentence-encoder model
2024-11-15 12:34:00,494 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2024-11-15 12:34:00,733 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2024-11-15 12:34:00,882 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2024-11-15 12:34:00,887 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


126: brexit, tory, bitcoin, warns, watchdog, governor, accused, unveils, lawless, abuse, chaotic, sentenced, thieves, economy, financial, cryptocurrency, tuchel, pension, climate, crisis, problems, news, trump, re, scotland, police, poor, flee, bbc, tumour, gary, government, cctv, lineker, critical, sudan, storm, strike, penalty, not, dying, weeks, then, uk, experts, ban, longer, friday, suspended, yet
76: lineker, tuchel, mcdavid, rugby, soccer, nhl, football, penalty, jets, final, mccall, england, coaching, wins, jones, injury, wales, league, rfk, win, coach, scotland, team, won, draws, blitz, tennis, connor, scores, tyson, brazil, match, france, strikes, victory, defence, greatest, chancellor, alex, argentina, jr, mike, greece, suspended, loses, donald, clash, bbc, strike, gamble
54: trump, presidential, donald, infowars, election, president, political, democratic, brexit, elect, rfk, jr, kennedy, news, andrew, friday, celebrity, secretary, gary, lobbyists, leader, unveils, musk, am

In [ ]:
from haystack_integrations.components.generators.ollama import OllamaGenerator
llm = OllamaGenerator(model="llama3.2")
prompt = f"""
Given the below keywords, provide a short description of the topic they fit into.
Keywords: {topic_words[:10]}
Topic: 
"""

4

In [49]:
guardian = TheGuardian().get_documents()

for d in guardian[:10]:
    print(d.content)

Shadow chancellor warns Reeves over EU ties after Bank chief says Brexit harming economy – UK politics live
Middle East crisis live: ‘Heavy strike’ reported in Beirut after Israeli evacuation order
RFK Jr condemned as ‘clear and present danger’ after Trump nomination
Two newly discovered stone circles on Dartmoor boost ‘sacred arc’ theory
Pakistan military has no intention of cutting deal with jailed former prime minister Imran Khan – sources
Davina McCall to have brain surgery for ‘very rare’ benign tumour
Cop29 live: calls grow to ban fossil fuel lobbyists at summit as numbers soar
England fans ‘treated like animals’ in Greece, prompting investigations
No 10 apologises after meat and alcohol served at Diwali celebration
Snow and ice expected in Scotland and north of England, Met Office warns


In [ ]:

topic_sizes

array([93, 72, 52, 38])

In [57]:
def check_feed(url):
    articles = feedparser.parse(url)["entries"]
    print(len(articles))
    print(articles[0])

In [58]:
check_feed("https://feeds.skynews.com/feeds/rss/home.xml")

10
{'title': "Chancellor 'not satisfied' as UK economic growth slows", 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://feeds.skynews.com/feeds/rss/home.xml', 'value': "Chancellor 'not satisfied' as UK economic growth slows"}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://news.sky.com/story/uk-economy-grows-by-01-between-july-and-september-but-slower-than-expected-13254485'}, {'length': '0', 'type': 'image/jpeg', 'href': 'https://e3.365dm.com/24/10/1920x1080/skynews-cash-money_6733362.jpg?20241030171824', 'rel': 'enclosure'}], 'link': 'https://news.sky.com/story/uk-economy-grows-by-01-between-july-and-september-but-slower-than-expected-13254485', 'summary': 'The UK economy grew by 0.1% between July and September, according to the Office for National Statistics (ONS).', 'summary_detail': {'type': 'text/html', 'language': None, 'base': 'https://feeds.skynews.com/feeds/rss/home.xml', 'value': 'The UK economy grew by 0.1% between July and Sep